# Note on Hertz source implementation
2021.02.21 Kurama Okubo

This notebook describes the theory of source time function of ball drop test following

McLaskey, G.C., Glaser, S.D. Acoustic Emission Sensor Calibration for Absolute Source Measurements. J Nondestruct Eval 31, 157–168 (2012). https://doi.org/10.1007/s10921-012-0131-2

and its implementation in OpenSWPC.

## Theory of Hertzian source

Hertzian theory gives analytical solution of ball impact. From the equation 7 in McLaskey and Glaser (2012), it is written as follows:

$$
f(t) = \begin{cases}
f_{max} \left\{\sin (\pi t / t_c) \right\} ^ {3/2} & 0 \leq |t| \leq t_c,\\
0 & \text{otherwise,}\\
\end{cases}
$$

where the contact time $t_c$ is

$$
t_c = 4.53( 4\rho_1 \pi (\delta_1 + \delta_2) /3 )^{2/5} R_1 v_0 ^{-1/5}.
$$

$\rho_1$ is the density, $R_1$ is the radius and $v_0$ is the velocity of incoming ball.

The maximum force $ f_{max} $ is

$$
f_{max} = 1.917\rho_1 ^{3/5} (\delta_1 + \delta_2) ^{-2/5} R_1 ^2 v_0 ^{6/5},
$$

$\delta_i$ is defined as

$$
\delta_i = (1-\mu_i^2)(\pi E_i),
$$

where $E$ and $\mu$ are the Young's modulus and Poisson's ratio, respectively. The subscripts 1 and 2 indicate the material properties of ball and the base specimen, respectively.

## Typical values of constants

### Steel Ball

| property | description  | value |
|:------:|:------| :------|
|  $\rho$| density of ball       | 7781.1 $\pm$ 2.3 $^{*1}$ [kg/m^3] (7850$^{*2}$)|
|  $R$  | radius of ball         | 1.5e-3 [m] |
|  $h$  | height of drop         | 500 [mm] |
|  $E$  | Young's modulus of ball| 209 $^{*2}$ [GPa]|
|  $\nu$| Poisson's ratio of ball| 0.29 $^{*2}$ |

* Measurement at NIED.
* $^{*2}$ McLaskey and Glaser 2010 Table 1.

### Rock block

| property | description  | value |
|:------:|:------| :------|
|  $\rho$| density of rock | 2980 $^{*3}$ [kg/m$^3$] |
|  $c_p$| P wave velocity of rock | 6919 $^{*3}$ [m/s] |
|  $c_s$| S wave velocity of rock | 3631 $^{*3}$ [m/s] |
|  $E$  | Young's modulus of rock | 103 $^{*3}$  [GPa]|
|  $\nu$| Poisson's ratio of rock | 0.31 $^{*3}$ |

*3 Fukuyama et al. (2016)


## Strategy of implementation

In OpenSWPC, all source time functions (STF) are normalized so that $\int_0^\infty \dot{M}(t) dt = 1$. Computations are performed using this normalized STF for the sake of numerical stability. Then, when dumping the results, the magnitude is corrected with the normalization factor.

To follow this manner, we normalize STF of Hertz source model by its total impulse. However, the integration is too tricky to obtain analytical form of the normalization factor. So we derived an approximated scaling factor as shown in the next section, which is used to normalize the STF.

It is noteworthy is that **do not use Hertz STF for moment tensor source with M0 input** because the normalization factor is an approximation, so $\int_0^\infty f(t) dt$ is not identical to unity.

Use Hertz STF for the body force mode with constants associated with dropping ball.

### Mathematical formulation of impulse for Hertz source
Our goal here is to compute

$$\int_{0}^{t_c} f(t) dt = \int_{0}^{t_c} f_{max} \left\{\sin (\pi t / t_c) \right\} ^ {3/2} dt,$$
which can be used as normalization factor equivalent to M0 in OpenSWPC.

$$\int_{0}^{t_c} f_{max} \left\{\sin (\pi t / t_c) \right\} ^ {3/2} dt = f_{max} \frac{t_c}{\pi} \int_{0}^{\pi} \left\{\sin (\phi) \right\} ^ {3/2} d\phi $$

Let
$$ I = \int_{0}^{\pi} \left\{\sin (\phi) \right\} ^ {3/2} d\phi, $$

Then
$$ \int_{0}^{t_c} f_{max} \left\{\sin (\pi t / t_c) \right\} ^ {3/2} dt = f_{max} \frac{t_c}{\pi}  I. $$

With integration by parts,
$$I=\int_{0}^{\pi} \frac{\cos ^2 {\phi}} {2\sqrt{\sin{\phi}}} d\phi$$
$$=\int_{0}^{\pi} \frac{1}{2\sqrt{\sin{\phi}}} d\phi - \frac{1}{2}I$$

Thus
$$I=\frac{1}{3} \int_{0}^{\pi}\frac{1}{\sqrt{\sin{\phi}}}d\phi. $$

I couldn't find the value above, so computed numerically and found a value as follows:

$$\frac{1}{3} \int_{0}^{\pi}\frac{1}{\sqrt{\sin{\phi}}}d\phi=\frac{1}{3}*5.244115108583619$$

Thus
$$I = 1.748038369527873$$

Therefore, the normalization factor $M0$ is written as:

$$ M0 = 1.748 f_{max} \dfrac{t_c}{\pi}.$$

Note that if using the typical values, $t_c \simeq 1e-5$ and $f_{max} = 131.0$, $M0 = 7.29e-4$.

## Tips for code implementation

`m_source.F90` l218: $$M0 = \sqrt{f_x^2 +f_y^2 + f_z^2}$$ for body force mode.$

`m_source.F90` l335: $f_z$ is normalized by 'M0', which is corrected when output in `m_output.F90`.


## メモ
OpenSWPCでは桁落ち防止のためにSTFをM0で正規化して計算し、出力する段階でM0をかけて補正している。Hertzの震源関数を用いる場合でも正規化するのが理想的であるが、その積分値を解析的に求めるのが難しいため、本実装では上記のように数値積分による概算値を用いて正規化した。その結果、Hertz sourceの時間積分もOpenSWPCの他の震源関数と同様におおよそ1に正規化した状態で計算が実行されている。